#  setup environment

In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import zipfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

# extracting bangla dataset 

In [2]:
num_classes=84
np.random.seed(133)
data_root='.'
full_filename=os.path.join(data_root, 'BanglaLekha-Isolated.zip')


def maybe_extract(filename, force=False):
    root=os.path.splitext(os.path.splitext(filename)[0])[0]
    root=root+'/Images' #as the classesdata are in images folder
    if os.path.isdir(root)and not force:
        print('%s already present - Skipping extraction of %s' %(root, filename))
    else:
        print('extracting data %s.this may take a while .please wait.' %root)
        zip=zipfile.ZipFile(filename)
        sys.stdout.flush()
        zip.extractall(data_root)
        zip.close()
    data_folders = [
        os.path.join(root, d)for d in sorted(os.listdir(root)) #listing all the classes directory
        if os.path.isdir(os.path.join(root, d))]
    if len(data_folders) !=num_classes: #checks lengths
        raise Exception(
            'Expected %d folders , one per class. found %d instead.' % (
                num_classes,len(data_folders)))
    print(data_folders)
    return data_folders

full_folders=maybe_extract(full_filename)
        

extracting data ./BanglaLekha-Isolated/Images.this may take a while .please wait.
['./BanglaLekha-Isolated/Images/1', './BanglaLekha-Isolated/Images/10', './BanglaLekha-Isolated/Images/11', './BanglaLekha-Isolated/Images/12', './BanglaLekha-Isolated/Images/13', './BanglaLekha-Isolated/Images/14', './BanglaLekha-Isolated/Images/15', './BanglaLekha-Isolated/Images/16', './BanglaLekha-Isolated/Images/17', './BanglaLekha-Isolated/Images/18', './BanglaLekha-Isolated/Images/19', './BanglaLekha-Isolated/Images/2', './BanglaLekha-Isolated/Images/20', './BanglaLekha-Isolated/Images/21', './BanglaLekha-Isolated/Images/22', './BanglaLekha-Isolated/Images/23', './BanglaLekha-Isolated/Images/24', './BanglaLekha-Isolated/Images/25', './BanglaLekha-Isolated/Images/26', './BanglaLekha-Isolated/Images/27', './BanglaLekha-Isolated/Images/28', './BanglaLekha-Isolated/Images/29', './BanglaLekha-Isolated/Images/3', './BanglaLekha-Isolated/Images/30', './BanglaLekha-Isolated/Images/31', './BanglaLekha-Isola

# Convert images into npArray and save into pickle file with separate class


In [ ]:
image_size=28 #pixel width and height 
pixel_depth=255.0 #Number of levels per pixel
